# **CS-331: Introduction to Artificial Intelligence - Spring 2024**

# Assignment 3: Searches

### Deadline:  29th April 2024, 11:55 PM

### Instructions

1. The aim of the assignment is to familiarise you with code implementation of concepts covered in class.

2. All cells must be run once before submission and should $\color{orange}{\text{clearly display the results (graphs/plots/visualizations etc)}}$. Failure to do so will result in deduction of points.

5. Use procedural programming style and comment your code properly. The grading breakdown has $\color{orange}{\text{five}}$ points reserved for well-commented, modular code.

### Submission Instructions

1. You are required to upload your solutions on LMS in the Assignment tab by the deadline.

2. The zip file MUST contain your jupyter notebook file (.ipynb) and python script (.py) files.

3. **EACH** file should be named as "Name_RollNumber_PA $\color{orange}{\text{x}}$" where $\color{orange}{\text{x = 3}}$ for this assignment.

### Late Submission Policy

1. You can submit upto 2 days late with a 10% deduction penalty (from your final obtained marks) for each late day.
2. $\color{red}{\text{No submission}}$ will be accepted after the late days are over.
2. $\color{red}{\text{No submission}}$ will be accepted via email/Slack, unless announced by the course staff beforehand. It is your responsibility to stay updated with any announcements pertaining to the assignment.

### Plagiarism Policy
All parts of this assignment are to be done $\color{red}{\text{INDEPENDENTLY}}$. The course stuff will refer any case of plagiarism
from others or the internet immediately to the DC. If you are confused about what
constitutes plagiarism, it is your responsibility to consult with the TAs
in a timely manner.

### Vivas

The teaching staff reserves the right to conduct a viva for any student.  

### Notes
The required packages for this assignment are already present in the first cell.

If you are running the noteook on your PC/laptop, it is STRONGLY advised that you install [conda](https://conda.io/projects/conda/en/latest/user-guide/getting-started.html) and work in a conda environment for this assignment. This will go a long way in ensuring you do not get dependency conflicts and your system does not slow down.


### <b>Problem Description</b>

In this assignment, you are tasked with navigating through our prestigious LUMS (read "Harvard") University using three search algorithms: A*, Best-first search, and BFS. For the purpose of this assignment, you will be walking through "legal" paths (and not the shortcuts we all take through grass :P). <br/>

Please read through the below **starter code** (please read comments and use of the functions as these will be used later in your main function, or when you wish to test your implementations). <br/>

 We have also provided formulas for the different heuristic functions that need to be implemented and used.




### Import Libraries

In [ ]:
%pip install osmnx
import osmnx as ox
from IPython.display import IFrame
import networkx as nx
import folium
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

## Folium and osmnx: Interactive Geospatial Visualization and Analysis

Let us first learn about the functionality of some libraries used in the starter code.
## osmnx
**osmnx** is a Python package that facilitates retrieving and visualizing data from OpenStreetMap.

In this assignment, we are using osmnx to:

- **Get Road Network Data**: Fetch road network data for LUMS from OpenStreetMap (OSM). This data includes information about roads, intersections, and other elements of the network.  <br/>

- **Plot Road Networks**: Plot road networks directly onto Folium maps.

## Folium
**Folium** is a Python library that facilitates the creation of interactive maps directly from Python code. It's particularly useful for visualizing geographical data.

In this assignment, we are using Folium to:

- **Visualize Road Networks**: Plot road networks obtained from OpenStreetMap (OSM) onto an interactive map. You can see the layout of roads, intersections, and other features. <br/>

- **Customize Markers**: Add markers to the map to highlight specific locations, such as nodes in the road network. These markers can display additional information when clicked, like node IDs or names. <br/>

- **Display Shortest Paths**: Visualize shortest paths on the map. You can see the route between two points highlighted on the map, along with the nodes and edges that form the path. <br/>



### Road Network Data Retrieval

Note that we have set network_type to "walk" so all paths that can be walked according the map's definitions will be used. You can try other setting like "drive" and see how the path you get in the visualizations changes to just include all the roads in LUMS.

In [ ]:
# Get the road network data from OpenStreetMap
def get_road_network(location, distance):
    G = ox.graph_from_address(location, network_type='walk', dist=distance)
    return G

### Road Network Visualization Function

In [ ]:
# Function to visualize the road network using Folium
def visualize_road_network(G, location, distance):
    """
        G: Graph object representing the road network
        location: location for which the road network is to be visualized
        distance: distance (in meters) around the specified location to be visualized
    """
    # Get the center (latitude and longitude) coordinates of the location
    lat, lng = ox.geocode(location)

    # Create a folium map centered at the location with an initial zoom level of 12
    map_center = [lat, lng]
    map_osm = folium.Map(location=map_center, zoom_start=12)

    # Add the road network graph (represented by "G") to the folium map
    # This overlays the road network onto the map, showing roads, intersections, and other features
    ox.plot_graph_folium(G, graph_map=map_osm, popup_attribute='name', node_labels=True, edge_width=20)

    # Add customized markers for nodes to view node IDs etc upon click
    for node, data in G.nodes(data=True):
        folium.Marker(location=[data['y'], data['x']], popup=f"Node: {node}").add_to(map_osm)

    # Display the folium map inline
    display(map_osm)

### Shortest Path Visualization Function

In [ ]:
# Function to visualize the shortest path on the map
def visualize_path_folium(G, shortest_path, location, source_node, target_nodes, distance):
    """
        G: Graph object representing the road network
        shortest_path:  A list of node IDs representing the shortest path between the source and target nodes
        location: location around which the map is centered
        source_node:  ID of the source node
        target_nodes: list of ID(s) of target nodes
        distance: distance (in meters) between the source and target nodes
    """

    # Get the center (latitude and longitude) coordinates of the location
    lat, lng = ox.geocode(location)

    # Create a folium map centered at the location
    map_center = [lat, lng]
    map_osm = folium.Map(location=map_center, zoom_start=12)

    # Add the road network graph to the folium map
    ox.plot_graph_folium(G, graph_map=map_osm, node_labels=True, edge_width=20)

    # Add markers for the source and destination nodes (source node is marked in green, destination node is marked in red)
    folium.Marker(location=(G.nodes[source_node]['y'], G.nodes[source_node]['x']), icon=folium.Icon(color='green'), popup=f'Source<br>Distance: {distance:.2f} meters').add_to(map_osm)


    for target_node in target_nodes:
      folium.Marker(location=(G.nodes[target_node]['y'], G.nodes[target_node]['x']), icon=folium.Icon(color='red'), popup='Destination').add_to(map_osm)

    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    # Get the coordinates of the shortest path
    shortest_path_coords = []
    for i in range(len(shortest_path)-1):
        edge = (shortest_path[i], shortest_path[i+1], 0)
        edge_coords = gdf_edges.loc[edge]['geometry']
        shortest_path_coords.extend([(point[1], point[0]) for point in edge_coords.coords])

    # Add the shortest path to the map as a PolyLine
    folium.PolyLine(locations=shortest_path_coords, color='blue', weight=5).add_to(map_osm)

    # Display the folium map inline
    display(map_osm)

## Heuristic Functions

### Euclidean Distance
Euclidean distance, also known as straight-line distance or L2 distance, measures the straight-line distance between two points in Euclidean space. It is calculated using the formula:

$$
\text{Euclidean Distance} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$
where $$(x_1, y_1)$$ and $$(x_2, y_2)$$ are the coordinates of the two points.

### Manhattan Distance
Manhattan distance, also known as city block distance or L1 distance, measures the distance between two points in a grid-based system. It is calculated as the sum of the absolute differences of their coordinates:
$$
\text{Manhattan Distance} = |x_2 - x_1| + |y_2 - y_1|
$$

### Haversine Distance
Haversine distance is used to calculate the distance between two points on the surface of a sphere, such as the Earth, given their longitude and latitude. It is calculated using the Haversine formula:
 $$
a = \sin^2\left(\frac{\Delta \text{lat}}{2}\right) + \cos(\text{lat}_1) \cdot \cos(\text{lat}_2) \cdot \sin^2\left(\frac{\Delta \text{lon}}{2}\right)
$$

$$
c = 2 \cdot \text{atan2}\left(\sqrt{a}, \sqrt{1-a}\right)
$$

$$
\text{Haversine Distance} = R \cdot c
$$

where `R = 6371.0` is the radius of the Earth, and $$\Delta \text{lat}$$ and $$\Delta \text{lon}$$ are the differences in latitude and longitude between the two points, respectively.


#### Helpful code you can use for calculating latitude and longitude, and node distances 



In [ ]:
loc = "LUMS Lahore, Pakistan"
dist = 500

G = get_road_network(loc, dist)

# Print nodes information
for node, data in G.nodes(data=True):
    print(f"Node {node}: Latitude - {data['y']}, Longitude - {data['x']}")

# Print edges information
for u, v, data in G.edges(data=True):
    print(f"Edge ({u}, {v}): Length - {data['length']}")

## Task 0: Implement Heuristics

In [ ]:
# Implement these. Feel free to modify function arguments for compatibility with your code if needed.
def euclidean_heuristic(node1, node2):
    '''
    node1: The ID of the first node
    node2: The ID of the second node

    Returns the Euclidean distance between the two nodes
    '''
    pass

def manhattan_heuristic(node1, node2):
    '''
    node1: The ID of the first node
    node2: The ID of the second node

    Returns the Manhattan distance between the two nodes
    '''
    pass

def haversine_heuristic(node1, node2):
    '''
    node1: The ID of the first node
    node2: The ID of the second node

    Returns the Haversine distance between the two nodes
    '''
    pass

## Task 1A: A* Algorithm Implementation

Implement the A* algorithm to find the shortest path between a source node and a target node. Feel free to make helper functions if you want.

### Helpful Pointers
- You may use a priority queue to store nodes to be explored. Additionally, you may initialize dictionaries to keep track of the path and cost to reach each node from the start node.

- Once the goal node is reached, reconstruct the shortest path by backtracking through the dictionary, starting from the goal node and ending at the start node.

- Remember to track & return the total distance for use with the visualization functions later



In [ ]:
# Implement A* search algorithm
def a_star(graph, start, goal, heuristic_func):
    '''
    graph: The graph object representing the road network.
    start: The starting node ID.
    goal: The destination node ID.
    heuristic_func: The heuristic function to be used to calculate the cost.
    '''
    pass

## Task 2: Best-First Search Algorithm Implementation

Implement the Best-First Search algorithm to find a path between a source node and target node. Feel free to make helper functions if you want.


In [ ]:
# Implement Best-First Search algorithm
def best_first_search(graph, start, goal, heuristic_func=euclidean_heuristic):
    '''
    graph: The graph object representing the road network.
    start: The starting node ID.
    goal: The destination node ID.
    heuristic_func: Euclidean heuristic ONLY
    '''
    pass

## Task 3: Informed Breadth-First Search Implementation
In this task, you'll implement a twist on the vanilla Breadth-First Search (BFS) algorithm. BFS is a blind search algorithm, thus it explores all possibilities without considering the cost, but integrating heuristic information can make it better (thus an "informed" search algo). <br/>

Use the Euclidean heuristic with BFS in the implementation below.






In [ ]:
# Implement informed Breadth-First Search algorithm
def bfs(graph, start, goal, heuristic_func=euclidean_heuristic):
   '''
    graph: The graph object representing the road network.
    start: The starting node ID.
    goal: The destination node ID.
    heuristic_func: Euclidean heuristic ONLY
   '''
   pass

# Running the visualisations
Now it's time to find out the shortcuts in LUMS! Write a `main` function that will allow you to visualise the different algorithms using the starter functions provided at the beginning of the assignment.

### Steps
1. Build the initial graph network and visualise it once.

3. Choose the source node as `810005319` (SSE) and target node as `11337034500` (SDSB). You can view these node values in the map by hovering too!

4. Get the shortest route and visualise it for EACH algo you have implemented. For A star, use all three heuristics and visualise. Note that you will need to send your target node in a list to the `visualize_path_folium` function (check its arguments above).

Note: You can change the source and target for fun too, however we only expect you to visualize the paths from the provided source (SSE) and target (SDSB) destinations.

In [ ]:
# code here
location = "LUMS Lahore, Pakistan"
distance = 500
source = 810005319 # SSE
destination = 11337034500 # SDSB

# full map visualisation
# a_star visualisation with euclidean
# a_star visualisation with manhattan
# a_star visualisation with haversine
# best_first visualisation
# informed bfs visualisation

# A-star Search using NetworkX

This function utilizes the built-in A* algorithm provided by the NetworkX library to find the shortest path between two nodes in a graph. Use this along with the visualisation functions provided earlier to compare your a_star implementation if you want!!



In [ ]:
def astar_networkx_path(G, source, target):
  """
      G: Graph object representing the road network
      source:  ID of the source node
      target: ID of the target node
  """
  return nx.astar_path(G, source=source, target=target, weight='length')

## Task 4: Single Source And Multiple Destinations using A star


Now use your code from a_star implementation and modify it to go to multiple destinations in the most optimal way. Use the euclidean heuristic. Then visualize the shortest route obtained by your implementation using the`visualize_path_folium` function.

In [ ]:
# code here
def a_star_multiple(graph, start, goal, heuristic_func=euclidean_heuristic):
  pass

In [ ]:
# code for visualisation
source = 810005319 # SSE
dest1 =  # Cricket Ground (hover on map and find node value)
dest2 =  # Masjid hover on map and find node value)

# Analysis
Identify the algortihm which provided you with the shortest path. Compare it with the other graphs and explain why it was the best.

Answer here...